In [ ]:
import glob
import random
import base64
import pandas as pd
from PIL import Image
from io import BytesIO
from IPython.display import HTML

def get_image_data(folder_path):
    
    data = []
    for file_path in glob.glob(f"{folder_path}/*"):
        try:
            with Image.open(file_path) as img:
                file_name = file_path.split("/")[-1]
                color_mode = img.mode
                img_format = img.format
                channels = len(img.getbands())
                width, height = img.size
                file_size = round(
                    (img.fp.tell() / (1024 * 1024)), 2)
                full_path = file_path
                img.thumbnail((100, 100))
                buffer = BytesIO()
                img.save(buffer, format="PNG")
                img_base64 = base64.b64encode(buffer.getvalue()).decode("utf-8")

                data.append({
                    "File Name": file_name,
                    "Color Model": color_mode,
                    "Format": img_format,
                    "Channels": channels,
                    "File Size (MB)": file_size,
                    "Width": width,
                    "Height": height,
                    "Full Path": full_path,
                    "Thumbnail": f'<img src="data:image/png;base64,{img_base64}" width="150"/>'
                })
        except Exception:
            
            continue

    
    df = pd.DataFrame(data)
    return df


folder = r"C:\PythonImagesLab4"
df = get_image_data(folder)

HTML(df.to_html(escape=False))

In [ ]:
import os
from PIL import Image, ImageEnhance

def crop_to_square(image):
    width, height = image.size
    min_side = min(width, height)
    left = (width - min_side) // 2
    top = (height - min_side) // 2
    right = left + min_side
    bottom = top + min_side
    return image.crop((left, top, right, bottom))

def apply_color_filter(image, color):
    overlay = Image.new("RGB", image.size, color)
    return Image.blend(image, overlay, alpha=0.3)

def to_poster(folder_path, images_per_row, images_per_column):

    target_size = (500, 500)

    image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path)
                   if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif', 'jfif'))]

    if not image_files:
        raise ValueError("У папці немає зображень.")

    selected_images = random.choices(image_files, k=images_per_row * images_per_column)

    filters = ['red', 'green', 'blue', 'yellow', 'magenta', 'cyan']

    poster_width = images_per_row * target_size[0]
    poster_height = images_per_column * target_size[1]
    poster = Image.new('RGB', (poster_width, poster_height))

    for i, img_path in enumerate(selected_images):
        with Image.open(img_path) as img:
            img = crop_to_square(img)
            img = img.resize(target_size)
            color_filter = random.choice(filters)
            img = apply_color_filter(img, color_filter)

            x = (i % images_per_row) * target_size[0]
            y = (i // images_per_row) * target_size[1]
            poster.paste(img, (x, y))

    return poster

poster_folder = r'C:\PythonImagesLab4'

In [ ]:
to_poster(poster_folder, 3, 2)

In [ ]:
to_poster(poster_folder, 4, 3)

In [ ]:
to_poster(poster_folder, 5, 4)

In [ ]:
to_poster(poster_folder, 6, 5)